In [12]:
# Import and load the data file
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [19]:
# Preprocess data
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        #add documents in the corpus
        documents.append((w, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [28]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

88 documents
4 classes ['goodbye', 'greeting', 'options', 'thanks']
40 unique lemmatized words ["'s", ',', 'anyone', 'are', 'awesome', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'for', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'here', 'hey', 'hi', 'hola', 'how', 'is', 'later', 'me', 'next', 'nice', 'provide', 'see', 'thaanks', 'thank', 'thanks', 'that', 'there', 'time', 'until', 'what', 'with', 'you']


In [47]:
# Create training and testing data

# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

# output is a '0' for each tag and '1' for current tag (for each pattern)
output_row = list(output_empty)
output_row[classes.index(doc[1])] = 1

training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [52]:
# Build the model

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
1/1 [==============================] - 0s 79ms/step - loss: 1.4272 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 996us/step - loss: 1.0428 - accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 993us/step - loss: 1.3971 - accuracy: 0.0000e+00
Epoch 4/200
1/1 [==============================] - 0s 996us/step - loss: 0.9771 - accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 2ms/step - loss: 1.3439 - accuracy: 0.0000e+00
Epoch 6/200
1/1 [==============================] - 0s 996us/step - loss: 1.3735 - accuracy: 0.0000e+00
Epoch 7/200
1/1 [==============================] - 0s 998us/step - loss: 0.8258 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 0us/step - loss: 0.5673 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 999us/step - loss: 0.7127 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 2ms/step - loss: 0.4103 - accuracy: 1.0000


Epoch 82/200
1/1 [==============================] - 0s 0us/step - loss: 4.7684e-07 - accuracy: 1.0000
Epoch 83/200
1/1 [==============================] - 0s 998us/step - loss: 2.3842e-07 - accuracy: 1.0000
Epoch 84/200
1/1 [==============================] - 0s 999us/step - loss: 0.0017 - accuracy: 1.0000
Epoch 85/200
1/1 [==============================] - 0s 958us/step - loss: 5.2736e-04 - accuracy: 1.0000
Epoch 86/200
1/1 [==============================] - 0s 997us/step - loss: 3.7794e-04 - accuracy: 1.0000
Epoch 87/200
1/1 [==============================] - 0s 996us/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 88/200
1/1 [==============================] - 0s 996us/step - loss: 0.0067 - accuracy: 1.0000
Epoch 89/200
1/1 [==============================] - 0s 998us/step - loss: 0.0299 - accuracy: 1.0000
Epoch 90/200
1/1 [==============================] - 0s 997us/step - loss: 8.9296e-04 - accuracy: 1.0000
Epoch 91/200
1/1 [==============================] - 0s 997us/step - loss: 2.59

Epoch 162/200
1/1 [==============================] - 0s 998us/step - loss: 4.0531e-06 - accuracy: 1.0000
Epoch 163/200
1/1 [==============================] - 0s 998us/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 164/200
1/1 [==============================] - 0s 998us/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 165/200
1/1 [==============================] - 0s 997us/step - loss: 2.8610e-05 - accuracy: 1.0000
Epoch 166/200
1/1 [==============================] - 0s 997us/step - loss: 1.4186e-05 - accuracy: 1.0000
Epoch 167/200
1/1 [==============================] - 0s 998us/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 168/200
1/1 [==============================] - 0s 995us/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 169/200
1/1 [==============================] - 0s 0us/step - loss: 0.0035 - accuracy: 1.0000
Epoch 170/200
1/1 [==============================] - 0s 997us/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 171/200
1/1 [==============================] - 0s 997us

In [71]:
# Predict the response (Graphical User Interface)

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle

import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

def clean_up_sentence(sentence):
    
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [104]:
#Creating GUI with tkinter

import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Sara: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
base = Tk()
base.title("Sara The ChatBot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="20", width="300", font="Elephant",)
ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Elephant",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Elephant")

#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()